In [2]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

In [3]:
def MakeMesh():
    shape = MoveTo(0,0).RectangleC(20,20) \
        .MoveTo(0,1).RectangleC(5,0.5, "el1").Reverse() \
        .MoveTo(0,-1).RectangleC(5,0.5, "el2").Reverse() \
        .Face()
    shape.edges["el.*"].vertices.hpref=1
    mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=2))
    return shape,mesh

In [4]:
geo, mesh = MakeMesh()
Draw (geo)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [5]:
eps0 = 8.854e-12

def SolveProblem(mesh):
    fes = H1(mesh, order=5, dirichlet="el.*")
    u,v = fes.TnT()

    gfu = GridFunction(fes)
    gfu.Interpolate( mesh.BoundaryCF( {"el1":1, "el2":-1 }), mesh.Boundaries(".*"))

    a = BilinearForm(eps0*grad(u)*grad(v)*dx).Assemble()
    inv = a.mat.Inverse(freedofs=fes.FreeDofs())
    gfu.vec.data -= inv@a.mat * gfu.vec
    return gfu

gfu = SolveProblem(mesh)

ea = { "euler_angles" : [-70,0,-40]} 
Draw (gfu, deformation=True, scale=5, **ea);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'euler_angles': [-…

In [6]:
Draw (Norm(grad(gfu)), mesh, order=3, deformation=True, min=0, max=3, **ea);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'euler_angles': [-…

In [7]:
geo, mesh = MakeMesh()
mesh.RefineHP(3, 0.25)
gfu = SolveProblem(mesh)
Draw (gfu, deformation=True, scale=5, **ea)
Draw (Norm(grad(gfu)), mesh, order=3, deformation=True, min=0, max=3, **ea);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'euler_angles': [-…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'euler_angles': [-…

In [13]:
geo, mesh = MakeMeshRounded()
Draw(geo)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [10]:
gfu = SolveProblem(mesh)
Draw (gfu, deformation=True, scale=5, **ea)
Draw (Norm(grad(gfu)), mesh, order=3, deformation=True, min=0, max=2, **ea);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'euler_angles': [-…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'euler_angles': [-…

In [11]:
geo, mesh = MakeMeshRounded()
mesh.Curve(5)

Draw (mesh)
gfu = SolveProblem(mesh)
Draw (gfu, deformation=True, scale=5, **ea)
Draw (Norm(grad(gfu)), mesh, order=3, deformation=True, min=0, max=3, **ea);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'euler_angles': [-…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'euler_angles': [-…

## Exercise

In [ ]:
Compute the total energy in the electrostatic field 
 

In [18]:
def SolveElectroStatic(shape, order, refs, other_info=False):
    mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=3))
    for l in range(refs):
        mesh.Refine()

    fes = H1(mesh, order=order, dirichlet="el.*")
    gfu = GridFunction(fes)

    u, v = fes.TnT()
    eps0 = 8.854e-12
    a = BilinearForm(eps0*grad(u)*grad(v)*dx).Assemble()
    f = LinearForm(fes).Assemble()

    bc = mesh.BoundaryCF({"el1": 1, "el2": -1}, default=0)
    gfu.Set(bc, definedon=mesh.Boundaries("el.*"))

    gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * (f.vec - a.mat * gfu.vec)

    energy = 0.5 * InnerProduct(a.mat*gfu.vec, gfu.vec)
    return gfu, energy
    
rect_el1 = MoveTo(-0.4,  0.3).Rectangle(0.8, 0.1)
rect_el1.edges.name = "el1"  # <--- name the boundary edges
el1 = rect_el1.Face()
rect_el2 = MoveTo(-0.4, -0.4).Rectangle(0.8, 0.1)
rect_el2.edges.name = "el2"  # <--- name the boundary edges
el2 = rect_el2.Face()
outer = MoveTo(-5, -5).Rectangle(10, 10).Face()
shape = outer - el1 - el2
order = 3
refs  = 1
gfu, energy = SolveElectroStatic(shape, order=order, refs=refs, other_info=True)
print("Computed electrostatic energy =", energy)


AttributeError: 'netgen.libngpy._NgOCC.WorkPlane' object has no attribute 'edges'

In [12]:
n = specialcf.normal(2)
D = eps0 * grad(gfu)
sigma = D*n   # i.e. normal flux

Q_el1 = Integrate(sigma, mesh, definedon=mesh.Boundaries("el1"))
print("Total charge on electrode el1 =", Q_el1)

Q_el2 = Integrate(sigma, mesh, definedon=mesh.Boundaries("el2"))
print("Total charge on electrode el2 =", Q_el2)


Total charge on electrode el1 = -9.194763738508046e-43
Total charge on electrode el2 = 7.457520084934101e-44
